In [ ]:
#AWS cred locations
dbutils.fs.ls("/FileStore/tables")

Out[22]: [FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv', name='authentication_credentials.csv', size=202, modificationTime=1687110999000)]

Read Credentials

In [ ]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")


Mount S3

In [ ]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-12853887c065-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/mount_name"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
#dbutils are not supported outside of databricks notebook
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME) # RUN ONCE ONLY!

display(dbutils.fs.ls("/mnt/mount_name/topics/12853887c065.geo/partition=0/"))

dbutils.fs.unmount("/mnt/mount_name") #unmount if it is already mounted

Hadoop config and streaming from S3

In [ ]:
# Adding the packages required to get data from S3  
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages com.amazonaws:aws-java-sdk-s3:1.12.490,org.apache.hadoop:hadoop-aws:3.3.1 pyspark-shell"

# Configure the hadoop setting to read from the S3 bucket
hadoopConf.set('fs.s3a.access.key', ACCESS_KEY)
hadoopConf.set('fs.s3a.secret.key', SECRET_KEY)
hadoopConf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') # Allows the package to authenticate with AWS

# Stream from the S3 bucket
s3_bucket_url = "s3n://user-12853887c065-bucket/topics/12853887c065"
extension = "/partition=0/12853887c065"
while True:
    df_pin = spark.read.json(f"{s3_bucket_url}.pin{extension}.pin+0+*.json")
    df_geo = spark.read.json(f"{s3_bucket_url}.geo{extension}.geo+0+*.json")
    df_user = spark.read.json(f"{s3_bucket_url}.user{extension}.user+0+*.json")
    df_pin.show(5, True)
    df_geo.show(5, True)
    df_user.show(5, True)